## Download Dataset

In [1]:
dir_path = 'wind_dataset_excel'
file_names = ['2020.xlsx', '2021.xlsx', '2022.xlsx', '2023.xlsx']

In [2]:
import os
import pandas as pd

df_list = []

for file_name in file_names:
    file_path = os.path.join(dir_path, file_name)
    df = pd.read_excel(file_path)
    df_list.append(df)

In [3]:
df_list[0].head()

,연료원,지역,거래일,시간,0분,5분,10분,15분,20분,25분,30분,35분,40분,45분,50분,55분
0,풍력,강원도,2020-01-01,0,3024.0,4896.0,5040.0,4896.0,4896.0,4896.0,4896.0,5040.0,5136.0,5760.0,6720.0,6576.0
1,풍력,강원도,2020-01-01,1,6576.0,6240.0,4896.0,5760.0,5076.0,5040.0,4896.0,5040.0,5040.0,4896.0,5040.0,5040.0
2,풍력,강원도,2020-01-01,2,5760.0,5040.0,5040.0,4896.0,4896.0,4896.0,5040.0,4896.0,4896.0,5040.0,5040.0,4896.0
3,풍력,강원도,2020-01-01,3,5040.0,5040.0,4896.0,4896.0,5040.0,4896.0,4896.0,4896.0,4896.0,5040.0,8496.0,4896.0
4,풍력,강원도,2020-01-01,4,5040.0,4896.0,8640.0,4896.0,4896.0,5040.0,8496.0,5040.0,4896.0,5040.0,4896.0,8640.0


## Extract Jeju and Create new column named '분'

In [4]:
for i in range(len(df_list)):
    df = df_list[i].copy()
    df = df[df['지역'] == '제주도']

    new_df = pd.DataFrame(columns = ['Date', 'Hour', 'Min', 'Wind Power (Wh)'])
    for r in range(len(df)):
        for c in range(4, df.shape[1]):
            new_df.loc[len(new_df)] = [df.iloc[r, 2], df.iloc[r, 3], 5 * (c-4), df.iloc[r, c]]

    display(new_df)

    df_list[i] = new_df


,Date,Hour,Min,Wind Power (Wh)
0,2020-01-01,0,0,2580.0
1,2020-01-01,0,5,12720.0
2,2020-01-01,0,10,1740.0
3,2020-01-01,0,15,2940.0
4,2020-01-01,0,20,10980.0
...,...,...,...,...
105403,2020-12-31,23,35,1440.0
105404,2020-12-31,23,40,2640.0
105405,2020-12-31,23,45,2592.0
105406,2020-12-31,23,50,12240.0


,Date,Hour,Min,Wind Power (Wh)
0,2021-01-01,0,0,10992.0
1,2021-01-01,0,5,2640.0
2,2021-01-01,0,10,12240.0
3,2021-01-01,0,15,2640.0
4,2021-01-01,0,20,12240.0
...,...,...,...,...
105115,2021-12-31,23,35,7200.0
105116,2021-12-31,23,40,2400.0
105117,2021-12-31,23,45,3600.0
105118,2021-12-31,23,50,5400.0


,Date,Hour,Min,Wind Power (Wh)
0,2022-01-01,0,0,3600.000000
1,2022-01-01,0,5,5400.000000
2,2022-01-01,0,10,3600.000000
3,2022-01-01,0,15,4200.000000
4,2022-01-01,0,20,6610.526316
...,...,...,...,...
105115,2022-12-31,23,35,1324.210526
105116,2022-12-31,23,40,1504.210526
105117,2022-12-31,23,45,10924.210530
105118,2022-12-31,23,50,1504.210526


,Date,Hour,Min,Wind Power (Wh)
0,2023-01-01,0,0,1504.210526
1,2023-01-01,0,5,1504.210526
2,2023-01-01,0,10,10924.210530
3,2023-01-01,0,15,1504.210526
4,2023-01-01,0,20,1324.210526
...,...,...,...,...
52123,2023-06-30,23,35,165990.526300
52124,2023-06-30,23,40,204511.157900
52125,2023-06-30,23,45,217682.526300
52126,2023-06-30,23,50,213426.947400


In [5]:
save_path = 'wind_dataset_csv'
save_file_names = ['2020.csv', '2021.csv', '2022.csv', '2023.csv']

for i in range(4):
    df_list[i].to_csv(os.path.join(save_path, save_file_names[i]), index = False)

In [6]:
for i in range(4):
    print(save_file_names[i], df_list[i].shape)

2020.csv (105408, 4)
2021.csv (105120, 4)
2022.csv (105120, 4)
2023.csv (52128, 4)


In [8]:
df_concat = pd.concat(df_list[1:], axis = 0)
df_concat.to_csv(os.path.join(save_path, '2021_2023_5mins.csv'), index = False)

In [14]:
df_concat_sum = df_concat.groupby(['Date', 'Hour'])['Wind Power (Wh)'].sum()
df_concat_sum = df_concat_sum.to_frame(name = 'Wind Power (MWh)').reset_index()
df_concat_sum['Wind Power (MWh)'] = df_concat_sum['Wind Power (MWh)'].apply(lambda x : x / 1000)

df_concat_sum.to_csv(os.path.join(save_path, '2021_2023_1hour.csv'), index = False)